<a href="https://colab.research.google.com/github/Nobu90/first-repository/blob/main/analysis/nobu/nobu_rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 로드 및 환경 세팅

### 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 패키지 설치

In [ ]:
!pip install nltk
!sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl
!python3 -m pip install --upgrade pip
!python3 -m pip install konlpy 
!pip install --upgrade py

### 패키지 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from konlpy.tag import Kkma
from konlpy.tag import Okt
from google.cloud import translate_v2 as translate
from googletrans import Translator

### API 권한 등록

In [8]:
! export GOOGLE_APPLICATION_CREDENTIALS='/content/drive/MyDrive/beaverworks/data/dulcet-asset-375808-8de8cb4af8fe.json'

### 데이터 불러오기

In [9]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# 공공 데이터
worksheet = gc.open('DATAGO_SEOUL_2022.MENU_EXPLN_INFO_KOREAN').sheet1
rows = worksheet.get_all_values()
df_seoul = pd.DataFrame.from_records(rows)

# 샘플 데이터
df_unos = pd.read_csv('/content/drive/MyDrive/beaverworks/data/sample_data_unos.csv')
df_odqueen = pd.read_csv('/content/drive/MyDrive/beaverworks/data/sample_data_oderqueen.csv')
df_sample = pd.read_excel('/content/drive/MyDrive/beaverworks/data/sample_data_2.xlsx', )

### 개별 샘플 데이터 확인

In [11]:
df_unos.head(2)

,affiliate_id,매장명,메뉴코드,카테고리명,메뉴명,가격
0,pokeallday_eg,포케올데이_시흥은계점,"1,295,195",#NAME?,메인토핑추가,"2,000"
1,pokeallday_eg,포케올데이_시흥은계점,"1,295,196",#NAME?,아보카도추가,"2,000"


In [12]:
df_odqueen.head(2)

,브랜드코드,브랜드명,점포번호,점포명,사업자번호,MENU_CD,MENU_FULL_NM,MENU_NM,판매가격
0,rakun,라쿤카페,1001,라쿤라떼17,349-02-01908,B0000109,르완다 마쿠바 풀리,르완다 마쿠바 풀리,"5,500"
1,rakun,라쿤카페,1001,라쿤라떼17,349-02-01908,B0000110,과테말라 세리토스,과테말라 세리토스,"4,500"


In [13]:
df_sample.head(2)

,상점코드,상점명,카테고리코드,카테고리명,상품코드,상품명,가격,사용여부
0,woki_sandwich,우키샌드위치_송정점,32112,커피/라떼,1590070,(?˙?˙?) 2배용량 사이즈업 아이스 카페라떼,8600.0,Y
1,wapi_mia,와피_미아역점,59171,세트메뉴,1021509,A.대만샌드위치+아메리카노,4000.0,Y


### 샘플 데이터 통합 및 확인

In [71]:
df_beaverworks = pd.DataFrame({
            'busi_id':df_odqueen['브랜드코드'],
            'busi_name':df_odqueen['브랜드명'],
            'menu':df_odqueen['MENU_FULL_NM'], 
            'price':df_odqueen['판매가격']})

df_beaverworks = df_beaverworks.append(pd.DataFrame({
            'busi_id':df_unos['affiliate_id'],
            'busi_name':df_unos['매장명'],
            'menu':df_unos['메뉴명'], 
            'price':df_unos['가격']}))

df_beaverworks = df_beaverworks.append(pd.DataFrame({
            'busi_id':df_sample['상점코드'],
            'busi_name':df_sample['상점명'],
            'menu':df_sample['상품명'], 
            'price':df_sample['가격']}))

#통합 데이터 확인
df_beaverworks

,busi_id,busi_name,menu,price
0,rakun,라쿤카페,르완다 마쿠바 풀리,"5,500"
1,rakun,라쿤카페,과테말라 세리토스,"4,500"
2,rakun,라쿤카페,에티오피아 모모라 구지 G1,"5,500"
3,rakun,라쿤카페,브라질 만치케이라 드 미나스,"5,500"
4,rakun,라쿤카페,오늘의 커피,"5,000"
...,...,...,...,...
45,adult_kimbab,어른김밥,---------Take out---------,0.0
46,adult_kimbab,어른김밥,--------- 전화 결제X ---------,0.0
47,55gallon_2,(주)담청에프엔비,(1입)TBL-벤토나이트 카본 모래_울트라오더 씰 12KG,17000.0
48,coffee_found,커피파운드,[S&H]RWANDA KARIMA\n살구 청을 달콤하게 우려낸 듯 깔끔한 커피,NaN


### 공공 데이터 정보 확인

In [70]:
print(df_seoul.info())
df_seoul.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873118 entries, 0 to 873117
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   menu_id        873118 non-null  object
 1   menu           873118 non-null  object
 2   menu_expln     873118 non-null  object
 3   main_category  873118 non-null  object
 4   sub_category   873118 non-null  object
 5   busi_id        873118 non-null  object
 6   busi_name      873118 non-null  object
 7   region         873118 non-null  object
 8   branch         873118 non-null  object
dtypes: object(9)
memory usage: 60.0+ MB
None


,menu_id,menu,menu_expln,main_category,sub_category,busi_id,busi_name,region,branch
0,메뉴(ID),메뉴명,메뉴설명(MENU_EXPLN),메뉴카테고리대분류명,메뉴카테고리소분류명,식당(ID),식당명,지역명,지점명
1,20022,용궁상,고기,한식,,10012,명동정,중구,


### 컬럼명 변경

In [18]:
df_seoul.rename(columns={0:'menu_id', 1:"menu", 2:"menu_expln", 3:"main_category", 4:"sub_category", 5:"busi_id", 6:"busi_name", 7:"region", 8:"branch"}, inplace=True)
df_seoul.head(2)

,menu_id,menu,menu_expln,main_category,sub_category,busi_id,busi_name,region,branch
0,메뉴(ID),메뉴명,메뉴설명(MENU_EXPLN),메뉴카테고리대분류명,메뉴카테고리소분류명,식당(ID),식당명,지역명,지점명
1,20022,용궁상,고기,한식,,10012,명동정,중구,


# 데이터 전처리

### 결측 값 및 고유 값 확인

In [67]:
# 결측 값 확인
print(df_seoul['menu'].isnull().sum())
print(df_seoul['busi_id'].isnull().sum())

# 고유 값 확인
print(df_seoul['menu'].nunique())
print(df_seoul['busi_id'].nunique())

0
0
336327
32251


In [23]:
df_seoul_menu = df_seoul['menu']

In [24]:
sr_seoul_menu_unique = df_seoul_menu.unique()
df_seoul_menu_unique = pd.DataFrame(sr_seoul_menu_unique.T, columns = ['original'])
df_seoul_menu_unique['replaced'] = sr_seoul_menu_unique
df_seoul_menu_unique['busi_id'] = df_seoul['busi_id']

In [25]:
# index 0~336326
df_seoul_menu_unique 

,original,replaced,busi_id
0,메뉴명,메뉴명,식당(ID)
1,용궁상,용궁상,10012
2,NEW수라상,NEW수라상,10012
3,귀빈상,귀빈상,10012
4,명동정식,명동정식,10012
...,...,...,...
336322,께사디야(vegetable/3pcs),께사디야(vegetable/3pcs),101135
336323,께사디야(shirimp/3pcs),께사디야(shirimp/3pcs),101136
336324,께사디야(only cheese/3pcs),께사디야(only cheese/3pcs),101130
336325,치미창가(beef),치미창가(beef),101120


In [26]:
df_seoul_menu_unique.loc[300000:320000]

,original,replaced,busi_id
300000,클라우드 브레이크 쁘띠쉬라,클라우드 브레이크 쁘띠쉬라,92087
300001,다렌버스 데드 암쉬라즈,다렌버스 데드 암쉬라즈,92096
300002,엘토키 프레스티지 까베르네 소비뇽(750ml),엘토키 프레스티지 까베르네 소비뇽(750ml),92087
300003,엘토키 프레스티지 샤르도네(750ml),엘토키 프레스티지 샤르도네(750ml),92096
300004,산타 안타레스 까베르네 소비뇽(750ml),산타 안타레스 까베르네 소비뇽(750ml),92087
...,...,...,...
319996,안성 장터국밥 반상,안성 장터국밥 반상,96854
319997,여수 꼬막비빔밥 반상,여수 꼬막비빔밥 반상,96858
319998,진부령 황태구이 반상,진부령 황태구이 반상,96861
319999,어린이 떡갈비 한상,어린이 떡갈비 한상,96854


# 외국어 확인

### 외국어 필터 함수 정의

In [27]:
# 패턴 인자
english_words_pattern = "[A-Za-z]"
chinese_words_pattern = "[一-龥]"
japanese_words_pattern = "[ぁ-ゔ]+|[ァ-ヴー]+[々〆〤]+/u"

# 영어 필터 함수
def filter_english_words(pattern, dataframe, column_name):
    english_words_mask = dataframe[column_name].apply(lambda x: bool(re.match(pattern, x)))
    english_words_df = dataframe[english_words_mask]
    return english_words_df

# 중국어 필터 함수
def filter_chinese_words(pattern, dataframe, column_name):
    chinese_words_mask = dataframe[column_name].apply(lambda x: bool(re.search(pattern, x)))
    chinese_words_df = dataframe[chinese_words_mask]
    return chinese_words_df

# 일본어 필터 함수
def filter_japanese_words(pattern, dataframe, column_name):
    japanese_words_mask = dataframe[column_name].apply(lambda x: bool(re.search(pattern, x)))
    japanese_words_df = dataframe[japanese_words_mask]
    return japanese_words_df

## 외국어 개수 확인

In [31]:
english_words_df = filter_english_words(english_words_pattern, df_seoul_menu_unique, "replaced")
english_words_df.value_counts

<bound method DataFrame.value_counts of                         original                  replaced busi_id
2                         NEW수라상                    NEW수라상   10012
23              Mouton Cadet Red          Mouton Cadet Red   10012
26            Mouton Cadet White        Mouton Cadet White   10012
43                   SET B(3인추천)               SET B(3인추천)   10015
44                   SET A(2인추천)               SET A(2인추천)   10015
...                          ...                       ...     ...
336095                     C모듬세트                     C모듬세트  101049
336307                  TACO SET                  TACO SET  101130
336308  BURRITO SET(medium size)  BURRITO SET(medium size)  101120
336309   BURRITO SET(large size)   BURRITO SET(large size)  101130
336310             ENCHILADA SET             ENCHILADA SET  101120

[16276 rows x 3 columns]>

In [32]:
chinese_words_df = filter_chinese_words(chinese_words_pattern, df_seoul_menu_unique, "replaced")
chinese_words_df.value_counts

<bound method DataFrame.value_counts of              original       replaced busi_id
167          민어전(10個)       민어전(10個)   10016
169           전복회(1尾)        전복회(1尾)   10016
171        전복버터구이(5尾)     전복버터구이(5尾)   10016
172        전복버터구이(3尾)     전복버터구이(3尾)   10016
183          복초밥(10個)       복초밥(10個)   10018
...               ...            ...     ...
335694         찰광어(中)         찰광어(中)  100902
335719  동태탕(中/저녁5시이후)  동태탕(中/저녁5시이후)  100902
335720  동태탕(大/저녁5시이후)  동태탕(大/저녁5시이후)  100920
335828        소류완자(中)        소류완자(中)  100920
335829        야채볶음(中)        야채볶음(中)  100902

[16736 rows x 3 columns]>

In [33]:
japanese_words_df = filter_japanese_words(japanese_words_pattern, df_seoul_menu_unique, "replaced")
japanese_words_df.value_counts

<bound method DataFrame.value_counts of                  original           replaced busi_id
56116          텐진라멘天神ラーメン         텐진라멘天神ラーメン   28789
56119              카라멘唐めん             카라멘唐めん   28789
56123          쇼유라멘醬油ラーメン         쇼유라멘醬油ラーメン   28789
56125             가라아게唐揚げ            가라아게唐揚げ   28789
56127            아게교자揚げ餃子           아게교자揚げ餃子   28789
56129           숙주볶음もやし炒め          숙주볶음もやし炒め   28789
56131            명란밥明太子ご飯           명란밥明太子ご飯   28789
56132          깨오이 ごまきゅうり         깨오이 ごまきゅうり   28789
56136              마제멘混ぜ麺             마제멘混ぜ麺   28791
56138           쿠로라멘黒ラーメン          쿠로라멘黒ラーメン   28791
58043            전복죽 アワビ粥           전복죽 アワビ粥   28844
88192           츠케멘R(並盛り）          츠케멘R(並盛り）   35981
88196           츠케멘L(中盛り）          츠케멘L(中盛り）   35979
88200          츠케멘XL(大盛り）         츠케멘XL(大盛り）   35981
88201    매운맛 츠케멘R(辛い 並盛り)   매운맛 츠케멘R(辛い 並盛り)   35996
88203    매운맛 츠케멘L(辛い 中盛り）   매운맛 츠케멘L(辛い 中盛り）   35961
88204   매운맛 츠케멘XL(辛い 大盛り)  매운맛 츠케멘XL(辛い 大盛り)   35981
88206 

# 분리 추출 및 제거

In [34]:
# 특수 기호 제거
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'/|·|_|♡|★|▶|★|☆|●|-|!|™|#', repl=r' ', regex=True)

# 괄호 안 내용 분리 추출 ['parentheses']
df_seoul_menu_unique['parentheses'] = df_seoul_menu_unique.replaced.str.extract(r'\(([^()]+)\)')
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique.replaced.str.replace(pat=r'\(([^()]+)\)', repl=r'', regex=True)

# 세트, 코스 여부 분리 추출 ['is_set']
df_seoul_menu_unique['is_set'] = df_seoul_menu_unique.replaced.str.extract(r'(세트|셋트|SET|Set|set|코스|COURSE|Course|course|모듬|샘플러)')
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique.replaced.str.replace(pat=r'(세트|셋트|SET|Set|set|코스|COURSE|Course|course|모듬|샘플러)', repl=r'', regex=True)

# 세트 번호 분리 추출 ['set_type']
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].apply(lambda x: re.sub('([A-H])([가-힣])', '\\1 \\2', x))
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].apply(lambda x: re.sub('([가-힣]([A-H]))', '\\1 \\2', x))
df_seoul_menu_unique['set_type'] = df_seoul_menu_unique.replaced.str.extract(r'(\b[A-H]\b|\b\d\b)')
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique.replaced.str.replace(pat=r'\b[A-H]\b|\b\d\b', repl=r'', regex=True)

# 옵션, 변경 여부 분리 추출 ['is_option']
df_seoul_menu_unique['is_option'] = df_seoul_menu_unique.replaced.str.extract(r'(추가|변경|선택)')
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique.replaced.str.replace(pat=r'(추가|변경|선택)', repl=r'', regex=True)

# 불용어 제거

### 불용어_일반

In [39]:
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'특선|특가|프리미엄|스페셜', repl=r'', regex=True)

df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'^추천', repl=r'', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'단품|메뉴', repl=r'', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'체험', repl=r'', regex=True)

df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'TAKEOUT|Takeout|takeout|포장', repl=r'', regex=True)

### 불용어_원산지(가나다순)

In [40]:
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(
    pat=r'국내산|국산|독일산|미국산|브라질산|영국산|이탈리아산|일본산|중국산|캐나다산|프랑스산|호주산', 
    repl=r'', regex=True)

### 사이즈 정보 분리 추출 및 제거

In [41]:
df_seoul_menu_unique['size'] = df_seoul_menu_unique.replaced.str.extract(
    r'(小|中|大|\d+人分|\d+人|\b소\b|\b중\b|\b대\b|\d+인분|\d+~+\d+인|\d+인|\d+호|\d+개|PET|\d+KG|\d+pcs|\d+ea|\d+cc|\d+ml|\d+L|\d+oz|d+g|\sS\b|\sR\b|\sL\b)')
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique.replaced.str.replace(
    pat=r'小|中|大|\d+人分|\d+人|\b소\b|\b중\b|\b대\b|\d+인분|\d+~+\d+인|\d+인|\d+호|\d+개|PET|\d+KG|\d+pcs|\d+ea|\d+cc|\d+ml|\d+L|\d+oz|\d+g|\sS\b|\sR\b|\sL\b', 
    repl=r'', regex=True)

# 사전 생성 및 변역

### 접속사 변경

In [42]:
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'&', repl=r' 그리고 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'+', repl=r' 그리고 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'\band\b', repl=r' 그리고 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'with|w/|,', repl=r' 그리고 ', regex=True)

### [영어 사전 변환]

In [93]:
# 알파벳 변환(대문자 > 소문자)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.lower()

# 영어 사전
english_replacement_dict = {"sparkling": "스파클링", "original": "오리지날", "chicken": "치킨", "mineral": "미네랄", "brunch": "브런치", "burrito": "부리또"}

# 변환
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].replace(english_replacement_dict, regex=True)

In [94]:
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'orange', repl=r' 오렌지 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'americano', repl=r' 아메리카노 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'dinner', repl=r' 저녁 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'bottle', repl=r' 병 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'coffee', repl=r' 커피 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'fresh', repl=r' 후레시 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'lunch', repl=r' 점심 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'white', repl=r' 화이트 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'salad', repl=r' 샐러드 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'adult', repl=r' 성인 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'glass', repl=r' 잔 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'steak', repl=r' 스테이크 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'pasta', repl=r' 파스타 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'juice', repl=r' 주스 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'latte', repl=r' 라떼 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'only', repl=r' 오직 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'wine', repl=r' 와인 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'iced', repl=r' 차가운 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'ice', repl=r' 아이스 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'shot', repl=r' 샷 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'hot', repl=r' 뜨거운 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'new', repl=r' 새로운 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'bbq', repl=r' 바베큐 ', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'red', repl=r' 레드 ', regex=True)

### [중국어(한문) 사전 변환]

In [95]:
# 중국어 사전
chinese_replacement_dict = {"韓牛": "한우", "名品": "명품", "生": "생", "辛": "매운", "王": "큰", "一": "1", "二": "2", "三": "3"}
# 변환
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].replace(chinese_replacement_dict, regex=True)

### [일본어 사전 변환]

In [96]:
# 일본어 사전
japanese_replacement_dict = {"ラーメン": "라멘"}
# 변환
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].replace(japanese_replacement_dict, regex=True)

### [음료수 사전 변환]

In [97]:
# 음료수 사전
drink_replacement_dict = {"coke": "콜라", "coka": "콜라", "redbull": "레드불", "perrier": "페리에"}

# 변환
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].replace(drink_replacement_dict, regex=True)

### [주류 사전 변환]

In [98]:
# 맥주 사전
beer_replacement_dict = {"\bhite": "하이트", "cass": "카스", "terra": "테라", "asahi": "아사히", "kloud": "클라우드", "corona": "코로나", "singha": "싱하"}

# 변환
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].replace(beer_replacement_dict, regex=True)

# 리큐르 사전
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'jagermeister', repl=r'예거마이스터', regex=True)

# 양주 - 위스키
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'bourbon', repl=r'버번', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'scotch', repl=r'스카치', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'ballantine', repl=r'발렌타인', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'johnnie walker', repl=r'조니 워커', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'jack daniel', repl=r'잭 다니엘', regex=True)

# 양주 - 보드카
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'absolute|absolut', repl=r'앱솔루트', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'finlandia', repl=r'핀란디아', regex=True)

# 양주 - 진
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'hendrick', repl=r'헨드릭스', regex=True)

# 양주 - 럼
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'barcardi|bacardi', repl=r'바카디', regex=True)

# 사이더 - 사과주
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'cider', repl=r'사이더', regex=True)

# 브랜드 통합
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(
    pat=r'하이트|카스|테라|아사히|클라우드|코로나|싱하|칭다오|칭따오|하이네켄|기네스|비어|beer', repl=r'맥주', regex=True)

df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'참이슬|처음처럼|진로|화요|한라산|서울의밤|soju', repl=r'소주', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'백세주|매화수|매취순|산사춘|문배술|솔송주|한산소곡주', repl=r'전통주', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'버번|스카치', repl=r'위스키', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'vodca|vodka|앱솔루트|핀란디아', repl=r'보드카', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'헨드릭스', repl=r'진', regex=True)
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].str.replace(pat=r'바카디', repl=r'럼', regex=True)

### 잔여 단어 번역_구글 API

In [ ]:
from google.colab import files
files.upload()

In [79]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'dulcet-asset-375808-8de8cb4af8fe.json'

client = translate.Client()
result = client.translate('こんにちは', target_language='ko')
print(result['translatedText'])

안녕하세요


### 장시간(20분이 지나도 불가) 소요됨 (실전에서 결정 필요)

In [ ]:
# 영어 > 한국어 변환기
#rows_to_translate = df_seoul_menu_unique[df_seoul_menu_unique['replaced'].str.contains(r'[A-Za-z]')]

#for i, row in rows_to_translate.iterrows():
    #text_to_translate = row['replaced']
    #translated_text = client.translate(text_to_translate, target_language='ko')['translatedText']
    #df_seoul_menu_unique.at[row.name, 'replaced'] = translated_text

###

In [91]:
## 중국어 > 한국어 변환기
rows_to_translate = df_seoul_menu_unique[df_seoul_menu_unique['replaced'].str.contains(r'[一-龥]')]

for i, row in rows_to_translate.iterrows():
    text_to_translate = row['replaced']
    translated_text = client.translate(text_to_translate, target_language='ko')['translatedText']
    df_seoul_menu_unique.at[row.name, 'replaced'] = translated_text

### 영어 최종 제거 및 확인

In [100]:
english_words_df = filter_english_words(english_words_pattern, df_seoul_menu_unique, "replaced")
english_words_df

,original,replaced,busi_id,parentheses,is_set,set_type,is_option,size
23,Mouton Cadet Red,mouton cadet 레드,10012,NaN,NaN,NaN,NaN,NaN
26,Mouton Cadet White,mouton cadet 화이트,10012,NaN,NaN,NaN,NaN,NaN
826,BEST천연버터허니베이컨라이스,best천연버터허니베이컨라이스,10036,NaN,NaN,NaN,NaN,NaN
832,BEST산더미홍합로제파스타,best산더미홍합로제파스타,10036,NaN,NaN,NaN,NaN,NaN
841,STAR불쇼볼케이노돈가스,star불쇼볼케이노돈가스,10036,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
336034,SHIRLEY TEMPLE(RASPBERRY),shirley temple,100997,RASPBERRY,NaN,NaN,NaN,NaN
336035,SHIRLEY TEMPLE(BLUEBERRY),shirley temple,100997,BLUEBERRY,NaN,NaN,NaN,NaN
336036,TENNENT,tennent,100997,NaN,NaN,NaN,NaN,NaN
336307,TACO SET,taco,101130,NaN,SET,NaN,NaN,NaN


### 중국어 최종 제거 및 확인

In [101]:
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].apply(lambda x: re.sub(r'[一-龥]', '', x))

In [102]:
chinese_words_df = filter_chinese_words(chinese_words_pattern, df_seoul_menu_unique, "replaced")
chinese_words_df

,original,replaced,busi_id,parentheses,is_set,set_type,is_option,size


### 일본어 최종 제거 및 확인

In [55]:
df_seoul_menu_unique['replaced'] = df_seoul_menu_unique['replaced'].apply(lambda x: re.sub(r'[\u3040-\u309f\u30a1-\u30ff]', '', x))

In [56]:
japanese_words_df = filter_japanese_words(japanese_words_pattern, df_seoul_menu_unique, "replaced")
japanese_words_df

,original,replaced,busi_id,parentheses,is_set,set_type,is_option,size


# 전처리 확인(최종 고유 값)

In [103]:
df_seoul_menu_unique['replaced'].value_counts()

                1742
                1033
맥주               531
아메리카노            521
카페라떼             332
                ... 
더 플라잉팬의 아침         1
스크램블 에그와 소시지       1
버섯 매콤 크림 파스타       1
문어 오일 파스타          1
스지간장조림             1
Name: replaced, Length: 197880, dtype: int64

In [105]:
df_seoul_menu_unique.loc[220000:240000]

,original,replaced,busi_id,parentheses,is_set,set_type,is_option,size
220000,일품진로/화요토닉세트,일품소주 소주토닉,71932,NaN,세트,NaN,NaN,NaN
220001,일품진로 소맥 세트,일품소주 소맥,71920,NaN,세트,NaN,NaN,NaN
220002,일품진로 소맥 잔술,일품소주 소맥 잔술,71944,NaN,NaN,NaN,NaN,NaN
220003,이네딧 담 생맥주,이네딧 담 생맥주,71929,NaN,NaN,NaN,NaN,NaN
220004,히비키 재패니스 하모니 잔술,히비키 재패니스 하모니 잔술,71944,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
239996,스터 프라이드 에그플랜트,스터 프라이드 에그플랜트,76279,NaN,NaN,NaN,NaN,NaN
239997,코코넛 커리 베지터블,코코넛 커리 베지터블,76262,NaN,NaN,NaN,NaN,NaN
239998,마 파 두부,마 파 두부,76280,NaN,NaN,NaN,NaN,NaN
239999,스위트&사워 포크,스위트 그리고 사워 포크,76227,NaN,NaN,NaN,NaN,NaN


# ---------

# 품사 태깅하기

In [106]:
sr_menu_name = df_seoul_menu_unique['replaced'].drop_duplicates()
df_menu_name = sr_menu_name.to_frame()
df_menu_name.rename(columns={'replaced':'sentence'}, inplace=True)
df_menu_name

,sentence
0,명
1,용궁상
2,새로운 수라상
3,귀빈상
4,명동정식
...,...
336310,enchilada
336316,따코 알 빠스톨
336318,하드 쉘 따코
336319,하드 쉘 따코


In [107]:
okt = Okt()

def pos_tagging(sentence):
    return okt.pos(sentence)

df_menu_name['pos_tagged'] = df_menu_name['sentence'].apply(pos_tagging)

df_menu_name

,sentence,pos_tagged
0,명,"[(명, Noun)]"
1,용궁상,"[(용궁, Noun), (상, Suffix)]"
2,새로운 수라상,"[(새로운, Adjective), (수라상, Noun)]"
3,귀빈상,"[(귀빈, Noun), (상, Suffix)]"
4,명동정식,"[(명동, Noun), (정식, Noun)]"
...,...,...
336310,enchilada,"[(enchilada, Alpha)]"
336316,따코 알 빠스톨,"[(따, Verb), (코, Noun), (알, Noun), (빠, Noun), (..."
336318,하드 쉘 따코,"[(하드, Noun), (쉘, Noun), (따, Verb), (코, Noun)]"
336319,하드 쉘 따코,"[(하드, Noun), (쉘, Noun), (따, Verb), (코, Noun)]"


# 업종 분류하기

In [ ]:
sr_replaced_unique = df_seoul_menu_unique['busi_id']
df_replaced_unique = pd.DataFrame(sr_replaced_unique.T, columns = ['busi_id'])
df_replaced_unique['busi_type'] = df_seoul_menu_unique.groupby('busi_id')['replaced'].transform(lambda x: ', '.join(x))

df_replaced_unique